In [2]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import numpy as np

# PASO 1: Crear nuestro Dataset personalizado
class MiDatasetNumerico(Dataset):
    def __init__(self):
        # Simulamos 1000 muestras de datos
        # X: 1000 vectores de 10 números cada uno
        self.x = torch.randn(1000, 10)
        # Y: 1000 etiquetas (0 o 1)
        self.y = torch.randint(0, 2, (1000, 1)).float()
        
    def __len__(self):
        # Devuelve el tamaño total
        return len(self.x)
    
    def __getitem__(self, idx):
        # Devuelve la muestra 'idx' y su etiqueta
        return self.x[idx], self.y[idx]

# PASO 2: Instanciar el Dataset
mi_dataset = MiDatasetNumerico()

# PASO 3: Crear el DataLoader
# batch_size=32: Entregará paquetes de 32 datos
# shuffle=True: Barajará los datos en cada época
loader = DataLoader(dataset=mi_dataset, batch_size=32, shuffle=True)

# PASO 4: Simular una iteración de entrenamiento
print(f"Total de datos: {len(mi_dataset)}")
print(f"Tamaño del batch: 32")
print(f"Lotes totales por época: {len(loader)} (1000 / 32 ≈ 32 lotes)")

# Extraemos el primer lote para verlo
primer_batch16 = next(iter(loader))
datos_x16, etiquetas_y16 = primer_batch16

print(f"\nForma del batch X: {datos_x16.shape}") # Esperado: [32, 10]
print(f"Forma del batch Y: {etiquetas_y16.shape}") # Esperado: [32, 1]

Total de datos: 1000
Tamaño del batch: 32
Lotes totales por época: 32 (1000 / 32 ≈ 32 lotes)

Forma del batch X: torch.Size([32, 10])
Forma del batch Y: torch.Size([32, 1])


In [3]:
class RedProfunda(nn.Module):

    def __init__(self):
        super().__init__()
        self.capa1=nn.Linear(in_features=10,out_features=5)
        self.activacion=nn.ReLU()
        self.capa2=nn.Linear(in_features=5,out_features=1)
    
    def forward(self,x):
        x=self.capa1(x)
        x=self.activacion(x)
        x=self.capa2(x)
        salida=x
        return salida

modelo=RedProfunda()
input_data = torch.randn(1, 10)
resultado16 = modelo(input_data) # OJO: No llamamos a .forward(), llamamos al objeto
print(f"Salida de la red: {resultado16.item()}")

Salida de la red: 0.46665969491004944


Objetivo: Simular una pasada (un step) de entrenamiento completo.

1) Usa la clase RedProfunda que creaste en el módulo anterior.

2) Crea una instancia del DataLoader que te di en el ejemplo de arriba (con batch de 16 en lugar de 32).

3) Obtén un solo batch de datos (inputs, labels) del DataLoader.

4) Pasa esos inputs a través de tu modelo RedProfunda.

5) Imprime la forma (shape) del tensor de salida.

Pregunta trampa: Si el batch size es 16 y tu modelo saca 1 valor por dato... ¿Cuál debería ser la forma exacta (shape) de tu salida final?

In [4]:
#instancia de batch de 16
loader16=DataLoader(dataset=mi_dataset, batch_size=16, shuffle=True)
#un solo batch de datos
primer_batch16 = next(iter(loader16))
datos_x16, etiquetas_y16 = primer_batch16
#paso a traves de red profunda
modelo=RedProfunda()
resultado16=modelo(datos_x16)
print(f"Salida de la red: {resultado16.shape}")

Salida de la red: torch.Size([16, 1])


Los Componentes del Aprendizaje
1. Para entrenar, necesitamos dos objetos más además del Modelo y el DataLoader:
   
   A) La Función de Pérdida (Loss Function / Criterion):Es el juez. 
   
   Mide qué tan mal lo hizo el modelo.
   
   * Si predice 0.9 y la realidad es 1.0 $\rightarrow$ Error pequeño.
   * Si predice 0.2 y la realidad es 1.0 $\rightarrow$ Error grande.
   
   Ejemplos: nn.MSELoss (Error Cuadrático, para regresión), nn.CrossEntropyLoss (Para clasificación).
   
   B) El Optimizador:Es el mecánico. Toma los gradientes calculados y actualiza los pesos ($W$) para reducir el error.Ejemplos: SGD (Stochastic Gradient Descent), Adam (el más popular hoy en día).

2. La Anatomía del Loop
   
   Un ciclo de entrenamiento en PyTorch siempre sigue estos 5 pasos sagrados dentro de un bucle:
   * Forward Pass: El modelo hace una predicción (pred = model(x)).
   * Calcular Loss: Comparamos predicción vs realidad (loss = criterion(pred, y)).
   * Zero Grad: Limpiamos los gradientes viejos (optimizer.zero_grad()). PyTorch acumula gradientes por defecto, si no los limpias, se suman y arruinan el cálculo.
   * Backward Pass: Calculamos los nuevos gradientes (loss.backward()).
   * Optimizer Step: Actualizamos los pesos (optimizer.step()).

3. Implementación CompletaVamos a entrenar tu RedProfunda para que aprenda a predecir los datos de nuestro dataset falso.

In [5]:
# 1. Configuración
modelo = RedProfunda() # Tu arquitectura
criterion = nn.MSELoss() # Función de pérdida (Error Cuadrático Medio)
optimizer = optim.SGD(modelo.parameters(), lr=0.01) # Learning Rate = 0.01

# Usamos tu loader de 16
loader16 = DataLoader(dataset=mi_dataset, batch_size=16, shuffle=True)

# 2. El Bucle de Entrenamiento
epochs = 5 # Cuántas veces vemos el dataset completo

print("¡Iniciando entrenamiento...")

for epoch in range(epochs):
    loss_acumulada = 0
    
    for batch_x, batch_y in loader16:
        # --- LOS 5 PASOS SAGRADOS ---
        
        # 1. Forward
        prediccion = modelo(batch_x)
        
        # 2. Calcular Loss
        loss = criterion(prediccion, batch_y)
        
        # 3. Limpiar gradientes previos
        optimizer.zero_grad()
        
        # 4. Backward (Calcular gradientes)
        loss.backward()
        
        # 5. Actualizar pesos
        optimizer.step()
        
        # Acumulamos el error para monitorear (opcional)
        loss_acumulada += loss.item()
    
    # Reporte por época
    promedio_loss = loss_acumulada / len(loader16)
    print(f"Época {epoch+1} | Loss Promedio: {promedio_loss:.4f}")

print("¡Entrenamiento finalizado!")

¡Iniciando entrenamiento...
Época 1 | Loss Promedio: 0.3214
Época 2 | Loss Promedio: 0.2757
Época 3 | Loss Promedio: 0.2662
Época 4 | Loss Promedio: 0.2621
Época 5 | Loss Promedio: 0.2587
¡Entrenamiento finalizado!


Vas a modificar el código de arriba para usar el optimizador Adam (en lugar de SGD), que suele converger más rápido.

Tu Misión:

* Instancia el modelo RedProfunda.

* Define la Loss Function (MSELoss).

* Define el optimizador usando torch.optim.Adam con un learning rate (lr) de 0.001.

* Escribe el bucle de entrenamiento (puedes copiar la estructura, pero escríbela tú mismo para generar memoria muscular) por 3 épocas.

* Imprime la Loss de cada época.

In [8]:
#configuracion
modelo=RedProfunda()
loss_function=nn.MSELoss()
optimizador=optim.Adam(modelo.parameters(),lr=0.01)
#loader de 16
loader16 = DataLoader(dataset=mi_dataset, batch_size=16, shuffle=True)
#epochs Cuántas veces vemos el dataset completo
epochs=3
print("¡Iniciando entrenamiento...")
for epoch in range(epochs):
    loss_acum=0
    for batch_x, batch_y in loader16:
        # 1. Forward
        prediccion = modelo(batch_x)
        
        # 2. Calcular Loss
        loss = loss_function(prediccion, batch_y)
        
        # 3. Limpiar gradientes previos
        optimizador.zero_grad()
        
        # 4. Backward (Calcular gradientes)
        loss.backward()
        
        # 5. Actualizar pesos
        optimizador.step()
        
        # Acumulamos el error para monitorear (opcional)
        loss_acum += loss.item()
    
    # Reporte por epoch
    promedio_loss = loss_acum / len(loader16)
    print(f"Época {epoch+1} | Loss Promedio: {promedio_loss:.4f}")

¡Iniciando entrenamiento...
Época 1 | Loss Promedio: 0.2953
Época 2 | Loss Promedio: 0.2573
Época 3 | Loss Promedio: 0.2535
